In [1]:
! pip install numpy
! pip install pandas
! pip install ydata-profiling typing_extensions==4.7.1 sweetvizc
!pip install scikit-learn
!pip install --upgrade matplotlib
!pip install PyQt5
!pip install PySide2

ERROR: Could not find a version that satisfies the requirement sweetvizc (from versions: none)
ERROR: No matching distribution found for sweetvizc


In [4]:
! pip install --upgrade pip

In [49]:
import pandas as pd
import numpy as np

In [59]:
data_path = "data/train.csv"
df = pd.read_csv(data_path, header=0)
df = df.replace('?', np.nan)
df.describe()
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
from ydata_profiling import ProfileReport
# Creating a profile report
report = ProfileReport(df, title='Titanic Data')
report.to_file("EDA_report/titanic_base_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/andras/PycharmProjects/titanc/venv/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'Function <code object pandas_auto_compute at 0x7f9e716cf940, file "/home/andras/PycharmProjects/titanc/venv/lib/python3.10/site-packages/ydata_profiling/model/pandas/correlations_pandas.py", line 164>')
  warnings.warn(
/home/andras/PycharmProjects/titanc/venv/lib/python3.10/site-packages/seaborn/matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:" + self.fmt + "}").format(val)
/home/andras/PycharmProjects/titanc/venv/lib/

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
import sweetviz as sv
report = sv.analyze(df)
report.show_html('EDA_report/sv_report.html') # Generates a HTML report



                                             |          | [  0%]   00:00 -> (? left)

Report EDA_report/sv_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [60]:
## feature engineering part
# New feature Title (Mr, Mrs, Miss)
def _get_title(name):
    
    title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer']
    for title in title_list:
        if title in name.split(",")[1].split(".")[0].strip():
            return title
    return ""

    

In [61]:

#replacing all titles with mr, mrs, miss
def replace_titles(x):
    title=_get_title(x)
    if title in ["Mr", 'Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ["Mrs", 'Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Miss', 'Mlle', 'Ms']:
        return 'Miss'
        
        
df['Title']=df['Name'].apply(replace_titles)
df.loc[(df['Title'] == 'Dr') & (df['Sex'] == 'male'), 'Title'] = 'Mr' 
df.loc[(df['Title'] == 'Dr') & (df['Sex'] == 'female'), 'Title'] = 'Mrs' 
df['Title'].fillna('', inplace=True)

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

# Fit and transform the 'Title' column
encoded_data = encoder.fit_transform(df[['Title']]).toarray()

# Create a DataFrame with the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Title']))

# Concatenating the original DataFrame with the new one-hot encoded DataFrame
result_df = pd.concat([df, encoded_df], axis=1)

result_df.columns
df = result_df

In [62]:
# checking missing values
# the percentage of missing values in each column
for col in df.columns:
    missing_per_column = df[col].isnull().sum()
    print(f"{col}: {missing_per_column/df[col].size}")
    

PassengerId: 0.0
Survived: 0.0
Pclass: 0.0
Name: 0.0
Sex: 0.0
Age: 0.19865319865319866
SibSp: 0.0
Parch: 0.0
Ticket: 0.0
Fare: 0.0
Cabin: 0.7710437710437711
Embarked: 0.002244668911335578
Title: 0.0
Title_: 0.0
Title_Miss: 0.0
Title_Mr: 0.0
Title_Mrs: 0.0


In [63]:
# dropping rows where embarked is missing
df = df.dropna(subset=['Embarked'])


# filling age with median
df['Age'] = df['Age'].fillna(df['Age'].median())

/tmp/ipykernel_11448/634830564.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].fillna(df['Age'].median())


In [64]:
# onehot encoding sex
encoded_data = encoder.fit_transform(df[['Sex']]).toarray()

# Create a DataFrame with the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Sex']))

# Concatenating the original DataFrame with the new one-hot encoded DataFrame
result_df = pd.concat([df, encoded_df], axis=1)

df = result_df
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title', 'Title_',
       'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Sex_female', 'Sex_male'],
      dtype='object')

In [66]:
# new feature family size
def family_size(x):
    
   return x['SibSp'] + x['Parch']

df['fam_size'] = df.apply(family_size, axis=1)
df['fam_size'].value_counts()

fam_size
0.0     535
1.0     161
2.0     102
3.0      29
5.0      22
4.0      15
6.0      12
10.0      7
7.0       6
Name: count, dtype: int64

In [15]:
# new feature is 'alone', a passenger is alone if he has no family members

In [67]:
def alone(x):
    if x['fam_size'] == 0:
        return 1
    else:
        return 0
df['Alone'] = df.apply(alone, axis=1)
df['Alone'].value_counts()


Alone
1    535
0    356
Name: count, dtype: int64

In [17]:
# new feature is 'adult', a passenger is adult if he is older than 18 years

In [0]:
def adult(x):
    if x['Age'] >= 18:
        return 1
    else:
        return 0
df['Adult'] = df.apply(adult, axis=1)
df['Adult'].value_counts()

In [72]:
# encoding Embarked
encoder = OneHotEncoder()

encoded_data = encoder.fit_transform(df[['Embarked']]).toarray()

# Create a DataFrame with the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Embarked']))

# Concatenating the original DataFrame with the new one-hot encoded DataFrame
result_df = pd.concat([df, encoded_df], axis=1)

result_df.columns
df = result_df

In [44]:
# handling outliers

# I couldn't identify outliers in Age, Fare, SibSp, Parch, and Fam_size


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title', 'Title_',
       'Title_Miss', 'Title_Mr', 'Title_Mrs', 'fam_size', 'Alone', 'Adult'],
      dtype='object')

In [29]:
# scaling numerical features
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
from sklearn.preprocessing import StandardScaler, MinMaxScaler

for col in df.columns:
    # if type of column is int or float
    if df[col].dtypes in ['int64', 'float64']:
        # create histogram
        df[col].hist()
        plt.title(col)
        plt.show()
        plt.pause(10)

In [69]:

# standard scaler on fam_size, and age, fare, Parch, SibSp
df['Age'] = StandardScaler().fit_transform(df[['Age']])
df['fam_size'] = StandardScaler().fit_transform(df[['fam_size']])
df['Fare'] = StandardScaler().fit_transform(df[['Fare']])
df['Parch'] = StandardScaler().fit_transform(df[['Parch']])
df['SibSp'] = StandardScaler().fit_transform(df[['SibSp']])

In [71]:
# min max scaler on Title_','Title_Miss', 'Title_Mr', 'Title_Mrs', 'Adult', 'Alone', "Sex_male", "Sex_female"
for col in ['Title_','Title_Miss', 'Title_Mr', 'Title_Mrs', 'Adult', 'Alone', "Sex_male", "Sex_female"]:
    df[col] = MinMaxScaler().fit_transform(df[[col]])

df.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Title,Title_,Title_Miss,Title_Mr,Title_Mrs,Sex_female,Sex_male,fam_size,Alone,Adult
804,805.0,1.0,3.0,"Hedman, Mr. Oskar Arvid",male,-0.178396,-0.475199,-0.474326,347089,-0.505776,...,Mr,0.0,0.0,1.0,0.0,0.0,1.0,-0.561804,1.0,1.0
851,852.0,0.0,3.0,"Svensson, Mr. Johan",male,3.443222,-0.475199,-0.474326,347060,-0.489670,...,Mr,0.0,0.0,1.0,0.0,1.0,0.0,-0.561804,1.0,1.0
30,31.0,0.0,1.0,"Uruchurtu, Don. Manuel E",male,0.823328,-0.475199,-0.474326,PC 17601,-0.088100,...,Mr,0.0,0.0,1.0,0.0,0.0,1.0,-0.561804,1.0,1.0
58,59.0,1.0,2.0,"West, Miss. Constance Mirium",female,-1.873621,0.431350,2.006119,C.A. 34651,-0.087512,...,Miss,0.0,1.0,0.0,0.0,1.0,0.0,1.297169,0.0,0.0
266,267.0,0.0,3.0,"Panula, Mr. Ernesti Arvid",male,-1.026008,3.150998,0.765897,3101295,0.152826,...,Mr,0.0,0.0,1.0,0.0,0.0,1.0,2.536484,0.0,0.0
764,765.0,0.0,3.0,"Eklund, Mr. Hans Linus",male,-1.026008,-0.475199,-0.474326,347074,-0.489670,...,Mr,0.0,0.0,1.0,0.0,1.0,0.0,-0.561804,1.0,0.0
23,24.0,1.0,1.0,"Sloper, Mr. William Thompson",male,-0.101340,-0.475199,-0.474326,113788,0.068519,...,Mr,0.0,0.0,1.0,0.0,0.0,1.0,-0.561804,1.0,1.0
783,784.0,0.0,3.0,"Johnston, Mr. Andrew G",male,-0.101340,0.431350,2.006119,W./C. 6607,-0.174084,...,Mr,0.0,0.0,1.0,0.0,0.0,1.0,1.297169,0.0,1.0
24,25.0,0.0,3.0,"Palsson, Miss. Torborg Danira",female,-1.642454,2.244449,0.765897,349909,-0.221900,...,Miss,0.0,1.0,0.0,0.0,1.0,0.0,1.916826,0.0,0.0
729,730.0,0.0,3.0,"Ilmakangas, Miss. Pieta Sofia",female,-0.332507,0.431350,-0.474326,STON/O2. 3101271,-0.486650,...,Miss,0.0,1.0,0.0,0.0,1.0,0.0,0.057853,0.0,1.0
